In [20]:
import verifiers as vf
from dotenv import load_dotenv

load_dotenv()

/Users/vijayravichander/Code/rl-agents-intro-will/rl-agents-intro-will/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [10]:
import requests
from markdownify import markdownify

def fetch(url: str) -> str:
    """ 
    Add Doc Strings
    """

    try:
        response = requests.get(url)

        if response.status_code == 200:

            response = requests.get(url)

            response.raise_for_status()

            return markdownify(response.text) 
    
    except Exception as e:
        return f"Error: {str(e)}"

markdown_text = fetch("https://hebbarskitchen.com/recipes/dessert-recipes/")

with open("main_page.md", "a") as f:
    f.write(markdown_text)

In [11]:
for page in range(2, 16):
    url = "https://hebbarskitchen.com/recipes/dessert-recipes/page/" + str(page) + "/"
    markdown_text = fetch(url)
    with open(f"page_{page}.md", "a") as f:
        f.write(markdown_text)

In [15]:
bounty_chocolate_bar_markdown = fetch("https://hebbarskitchen.com/bounty-chocolate-recipe-bounty-bar/")

In [17]:
from openai import OpenAI
import os

client = OpenAI(base_url = os.getenv("DEEPINFRA_API_LINK"), api_key = os.getenv("DEEPINFRA_API_KEY"));

In [31]:
system_prompt = """
You're an helpful assistant. Given the document, you are required to generated 5 question and answers related to the main item in the document.
* Keep the question and answer about 10 - 20 words each. 
* Make sure to specify the entire item name in the question.
* The questions should be only related to the main item and not the document in general. 
* Try to generate questions that can be answered using this document and not any general questions. 
* You need to give the question and answer in JSON format and <final_answer> </final_answer> tags.
* Add some questions and answers related to ingredients and measurements of them item.
<final_answer>
[
{
    "question" : "", 
    "answer" : ""
}, ...
]
</final_answer>
"""


response = client.chat.completions.create(
    model = "deepseek-ai/DeepSeek-V3-0324", 
    messages = [{"role": "system", "content": system_prompt}, 
    {"role": "user", "content": bounty_chocolate_bar_markdown}]
)

completion = response.choices[0].message.content

print(completion)

<final_answer>
[
  {
    "question": "What are the main ingredients required to make Bounty Chocolate Bars?",
    "answer": "The main ingredients for Bounty Chocolate Bars are milk, sugar, desiccated coconut, cream, and milk chocolate."
  },
  {
    "question": "How much sugar is needed for the Bounty Chocolate Bars recipe?",
    "answer": "The recipe requires 3/4 cup of sugar for the Bounty Chocolate Bars."
  },
  {
    "question": "What is the recommended amount of desiccated coconut for making Bounty Chocolate Bars?",
    "answer": "You need 2 cups of desiccated coconut for the Bounty Chocolate Bars."
  },
  {
    "question": "How long should the Bounty Chocolate Bars be refrigerated before coating with chocolate?",
    "answer": "Refrigerate the coconut bars for 1 hour before coating them with melted chocolate."
  },
  {
    "question": "What is the method used to melt chocolate for coating the Bounty Bars?",
    "answer": "The double boiler method is used to melt the milk chocolat

In [28]:
parser = vf.XMLParser(['final_answer'], answer_field="final_answer")

parsed_answer = parser.parse_answer(completion)

In [29]:
import json
qa = json.loads(parsed_answer.final_answer)

In [30]:
for q in qa:
    print(q)

{'question': 'What are the main ingredients needed to make the bounty chocolate recipe?', 'answer': 'The main ingredients are milk, sugar, desiccated coconut, cream, and milk chocolate.'}
{'question': 'How long should you refrigerate the bounty bars for them to set completely?', 'answer': 'Refrigerate the bounty bars for at least 1 hour for them to set completely.'}
{'question': 'What is the purpose of adding cream to the bounty chocolate recipe?', 'answer': 'Adding cream makes the coconut layer creamy and tasty.'}
{'question': 'How can you melt the chocolate for coating the bounty bars?', 'answer': 'Melt the milk chocolate using the double boiler method.'}
{'question': 'How long can homemade bounty chocolate bars be stored?', 'answer': 'Homemade bounty bars can be stored for a week in an airtight container.'}
